In [1]:
# https://www.kaggle.com/mathormad/inceptionv3-baseline-lb-0-379/code
# fork of scratch8, 29

In [2]:
%matplotlib inline
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle
import keras_metrics

import warnings
warnings.filterwarnings("ignore")
SIZE = 512

Using TensorFlow backend.


In [3]:
# https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric/notebook
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + (1 - K.mean(f1))

In [4]:
# Load dataset info
path_to_train = '../data/train/'
data = pd.read_csv('../data/train.csv')
# path_to_external_data = '../data/external_data/external_data_1/'
# edata = pd.read_csv('../data/external_data.csv')

In [5]:
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [6]:
data.shape

(31072, 2)

In [7]:
train_dataset_info = []

for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
    
# for name, labels in zip(edata['id'], edata['labels'].str.strip('[]')):
#     labels = labels.split(',')
#     train_dataset_info.append({
#         'path':os.path.join(path_to_external_data, name),
#         'labels':np.array([int(label) for label in labels])})
    
train_dataset_info = np.array(train_dataset_info)

In [8]:
train_dataset_info.shape

(31072,)

In [9]:
import threading

# class data_generator:
    
class threadsafe_iter:
    """
    Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return self.it.__next__()

def threadsafe_generator(f):
    """
    A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def create_train(dataset_info, batch_size, shape, augument=True):
    assert shape[2] == 3
    while True:
        dataset_info = shuffle(dataset_info)
        for start in range(0, len(dataset_info), batch_size):
            end = min(start + batch_size, len(dataset_info))
            batch_images = []
            X_train_batch = dataset_info[start:end]
            batch_labels = np.zeros((len(X_train_batch), 28))
            for i in range(len(X_train_batch)):
                image = load_image2(
                    X_train_batch[i]['path'], shape)
#                     image = tdi[i+start]
#                     image = cv2.resize(image, (shape[0], shape[1]))
                if augument:
                    image = augment2(image)

                batch_images.append(image/255.)
                batch_labels[i][X_train_batch[i]['labels']] = 1
            yield np.array(batch_images, np.float32), batch_labels

def load_image(path, shape):

    if len(path.split('/')[3]) != 36:
        print(path)
    image_red_ch = Image.open(path+'_red.png')
    image_yellow_ch = Image.open(path+'_yellow.png')
    image_green_ch = Image.open(path+'_green.png')
    image_blue_ch = Image.open(path+'_blue.png')
    image1 = np.stack((
        np.array(image_red_ch),
        np.array(image_green_ch), 
        np.array(image_blue_ch)), -1)
    if len(path.split('/')[3]) != 36:
        print(image1.shape)
    w, h = 512, 512
#         zero_data = np.zeros((h, w), dtype=np.uint8)
#         image2 = np.stack((
#             np.array(image_red_ch),
#             np.array(image_green_ch), 
#             np.array(image_yellow_ch)), -1)
#         image3 = np.stack((
#             np.array(image_yellow_ch),
#             np.array(image_green_ch), 
#             np.array(image_blue_ch)), -1)
# #         print(image1.shape, image2.shape)
#         image = np.vstack((image1, image2, image3))
#         print(image.shape)
    image =image1
#         image = canny_image4(image1)
    image = cv2.resize(image, (shape[0], shape[1]))
    if len(path.split('/')[3]) != 36:
        print(image.shape)
    return image

def load_image2(path, shape):
    colors = ['red','green','blue']
    flags = cv2.IMREAD_GRAYSCALE
    img = [cv2.imread(path+'_'+color+'.png', flags).astype(np.float32)
       for color in colors]
    return np.stack(img, axis=-1)


def augment2(image):
    augment_img = iaa.Sequential([
        iaa.OneOf([
            iaa.Affine(rotate=0),
            iaa.Affine(rotate=90),
            iaa.Affine(rotate=180),
            iaa.Affine(rotate=270),
            iaa.Fliplr(0.5),
            iaa.Flipud(0.5),
        ])], random_order=True)

    image_aug = augment_img.augment_image(image)
    return image_aug
def augment(image):
    augment_img = iaa.Sequential([
        iaa.OneOf([
                iaa.Fliplr(0.5), # horizontal flips
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Flipud(0.5),
                iaa.Crop(percent=(0, 0.1)), # random crops
                # Small gaussian blur with random sigma between 0 and 0.5.
                # But we only blur about 50% of all images.
                iaa.Sometimes(0.5,
                    iaa.GaussianBlur(sigma=(0, 0.5))
                ),
                # Strengthen or weaken the contrast in each image.
                iaa.ContrastNormalization((0.75, 1.5)),
                # Add gaussian noise.
                # For 50% of all images, we sample the noise once per pixel.
                # For the other 50% of all images, we sample the noise per pixel AND
                # channel. This can change the color (not only brightness) of the
                # pixels.
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                # Make some images brighter and some darker.
                # In 20% of all cases, we sample the multiplier once per channel,
                # which can end up changing the color of the images.
                iaa.Multiply((0.8, 1.2), per_channel=0.2),
                # Apply affine transformations to each image.
                # Scale/zoom them, translate/move them, rotate them and shear them.
                iaa.Affine(
                    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                    translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                    rotate=(-180, 180),
                    shear=(-8, 8)
                )
            ])], random_order=True)

    image_aug = augment_img.augment_image(image)
    return image_aug


In [10]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Lambda, multiply
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Input, Conv2D
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

In [11]:
def create_model(input_shape, n_out):
    in_lay = Input(input_shape)
    base_pretrained_model = ResNet50(input_shape =  input_shape, include_top = False, weights = 'imagenet')
    base_pretrained_model.trainable = False
    pt_depth = base_pretrained_model.get_output_shape_at(0)[-1]
    pt_features = base_pretrained_model(in_lay)
    from keras.layers import BatchNormalization
    bn_features = BatchNormalization()(pt_features)

    # here we do an attention mechanism to turn pixels in the GAP on an off

    attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(Dropout(0.5)(bn_features))
    attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
    attn_layer = Conv2D(8, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
    attn_layer = Conv2D(1, kernel_size = (1,1), padding = 'valid', activation = 'sigmoid')(attn_layer)
    # fan it out to all of the channels
    up_c2_w = np.ones((1, 1, 1, pt_depth))
    up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
                   activation = 'linear', use_bias = False, weights = [up_c2_w])
    up_c2.trainable = False
    attn_layer = up_c2(attn_layer)

    mask_features = multiply([attn_layer, bn_features])
    gap_features = GlobalAveragePooling2D()(mask_features)
    gap_mask = GlobalAveragePooling2D()(attn_layer)
    # to account for missing values from the attention model
    gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
    gap_dr = Dropout(0.25)(gap)
    dr_steps = Dropout(0.25)(Dense(128, activation = 'relu')(gap_dr))
    out_layer = Dense(n_out, activation = 'sigmoid')(dr_steps)
    model = Model(inputs = [in_lay], outputs = [out_layer])
    
    return model

In [12]:
# warm up model
model = create_model(
    input_shape=(SIZE,SIZE,3), 
    n_out=28)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 16, 16, 2048) 23587712    input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 2048) 8192        resnet50[1][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 16, 16, 2048) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (

In [13]:
model.compile(loss=f1_loss,
            optimizer=Adam(lr=1e-3),
            metrics=[f1])
# model.load_weights('../cache/R50-57-maximus.h5')

In [14]:
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

epochs = 10; batch_size = 16
checkpoint = ModelCheckpoint('../cache/R50-57-maximus.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                                   verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=6)
callbacks_list = [checkpoint, early, reduceLROnPlat]


In [15]:
# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
np.random.shuffle(indexes)
train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)


In [16]:
batch_size = 16


# create train and valid datagens
# train_generator = data_generator.create_train(
#     train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
# validation_generator = data_generator.create_train(
#     train_dataset_info[valid_indexes], 16, (SIZE,SIZE,3), augument=True)

train_generator = create_train(
    train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
validation_generator = create_train(
    train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=True)


In [17]:
for layer in model.layers:
    layer.trainable = True
model.layers[0].trainable = False
model.layers[1].trainable = False
model.layers[2].trainable = False


In [18]:
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=2, 
    verbose=1)

Epoch 1/2
1651/1651 [==============================] - 551s 334ms/step - loss: 1.0309 - f1: 0.1277 - val_loss: 1.2087 - val_f1: 0.0000e+00
Epoch 2/2
1651/1651 [==============================] - 526s 319ms/step - loss: 0.9839 - f1: 0.1651 - val_loss: 1.2263 - val_f1: 0.0000e+00


In [24]:
# train all layers
epochs=120
for layer in model.layers:
    layer.trainable = True
model.compile(loss=f1_loss,
            optimizer=Adam(lr=1e-4),
            metrics=[f1])
# model.fit_generator(
#     train_generator,
#     steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
#     validation_data=validation_generator,
#     validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
#     epochs=epochs, 
#     verbose=1,
#     workers=10,
#     callbacks=callbacks_list)

In [25]:
batch_size = 8


# create train and valid datagens
# train_generator = data_generator.create_train(
#     train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
# validation_generator = data_generator.create_train(
#     train_dataset_info[valid_indexes], 16, (SIZE,SIZE,3), augument=True)

train_generator = create_train(
    train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
validation_generator = create_train(
    train_dataset_info[valid_indexes], 16, (SIZE,SIZE,3), augument=True)


model.fit_generator(
    generator=train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/120
3302/3302 [==============================] - 1247s 378ms/step - loss: 0.9836 - f1: 0.1480 - val_loss: 0.8872 - val_f1: 0.2336

Epoch 00001: val_loss improved from inf to 0.88720, saving model to ../cache/R50-57-maximus.h5
Epoch 2/120
3302/3302 [==============================] - 1237s 375ms/step - loss: 0.9469 - f1: 0.1702 - val_loss: 0.8900 - val_f1: 0.2363

Epoch 00002: val_loss did not improve from 0.88720
Epoch 3/120
3302/3302 [==============================] - 1234s 374ms/step - loss: 0.9325 - f1: 0.1780 - val_loss: 0.8545 - val_f1: 0.2582

Epoch 00003: val_loss improved from 0.88720 to 0.85455, saving model to ../cache/R50-57-maximus.h5
Epoch 4/120
3302/3302 [==============================] - 1230s 373ms/step - loss: 0.9222 - f1: 0.1842 - val_loss: 0.8575 - val_f1: 0.2614

Epoch 00004: val_loss did not improve from 0.85455
Epoch 5/120
3302/3302 [==============================] - 1231s 373ms/step - loss: 0.9153 - f1: 0.1888 - val_loss: 0.8492 - val_f1: 0.2668

Epoch 000


Epoch 00038: val_loss did not improve from 0.79505
Epoch 39/120
3302/3302 [==============================] - 1198s 363ms/step - loss: 0.7971 - f1: 0.2495 - val_loss: 0.8011 - val_f1: 0.3005

Epoch 00039: val_loss did not improve from 0.79505
Epoch 40/120
3302/3302 [==============================] - 1198s 363ms/step - loss: 0.7980 - f1: 0.2492 - val_loss: 0.8017 - val_f1: 0.2985

Epoch 00040: val_loss did not improve from 0.79505

Epoch 00040: ReduceLROnPlateau reducing learning rate to 1.0000000116860975e-08.


In [26]:
# Create submit
from tqdm import tqdm_notebook
submit = pd.read_csv('../data/sample_submission.csv')
predicted = []
draw_predict = []
# model = create_model(
#     input_shape=(SIZE,SIZE,3), 
#     n_out=28)
# for layer in model.layers:
#     layer.trainable = True
# model.compile(loss=f1_loss,
#             optimizer=Adam(lr=1e-4),
#             metrics=[f1])
model.load_weights('../cache/R50-57-maximus.h5')
for name in tqdm_notebook(submit['Id']):
    path = os.path.join('../data/test/', name)
    image = load_image2(path, (SIZE,SIZE,3))/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.5]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted

HBox(children=(IntProgress(value=0, max=11702), HTML(value='')))

In [27]:
submit.to_csv('../submissions/sub57-max-a.csv', index=False)

In [28]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub57-max-a.csv -m ""

from time import sleep
sleep(20)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 468k/468k [00:13<00:00, 36.7kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName                date                 description  status    publicScore  privateScore  
----------------------  -------------------  -----------  --------  -----------  ------------  
sub57-max-a.csv         2018-12-21 23:46:32               complete  0.455        None          
sub57-max.csv           2018-12-21 08:43:25               complete  0.000        None          
sub50-max-055.csv       2018-12-18 20:29:16               complete  0.510        None          
sub50-max-f.csv         2018-12-18 20:15:51               complete  0.513        None          
sub50-max-f-1-l.csv     2018-12-18 16:40:56               complete  0.546        None          
sub50-max-f-1.csv       2018-12-18 16:38:26               complete  0.525        None          
sub50-max-f-1.csv       2018-12-18 15:53:31               complete  0.061        None

In [29]:
submit = pd.read_csv('../data/sample_submission.csv')
predicted = []
for ii, name in tqdm_notebook(enumerate(submit['Id'])):
    
    score_predict = draw_predict[ii]
    label_predict = np.arange(28)[score_predict>=0.45]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted
submit.to_csv('../submissions/sub57-max-b.csv', index=False)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [30]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub57-max-b.csv -m ""

from time import sleep
sleep(20)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 469k/469k [00:12<00:00, 39.0kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName                date                 description  status    publicScore  privateScore  
----------------------  -------------------  -----------  --------  -----------  ------------  
sub57-max-b.csv         2018-12-22 01:45:44               complete  0.456        None          
sub57-max-a.csv         2018-12-21 23:46:32               complete  0.455        None          
sub57-max.csv           2018-12-21 08:43:25               complete  0.000        None          
sub50-max-055.csv       2018-12-18 20:29:16               complete  0.510        None          
sub50-max-f.csv         2018-12-18 20:15:51               complete  0.513        None          
sub50-max-f-1-l.csv     2018-12-18 16:40:56               complete  0.546        None          
sub50-max-f-1.csv       2018-12-18 16:38:26               complete  0.525        None

In [31]:
submit = pd.read_csv('../data/sample_submission.csv')
predicted = []
for ii, name in tqdm_notebook(enumerate(submit['Id'])):
    
    score_predict = draw_predict[ii]
    label_predict = np.arange(28)[score_predict>=0.40]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted
submit.to_csv('../submissions/sub57-max-c.csv', index=False)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [32]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub57-max-c.csv -m ""

from time import sleep
sleep(20)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 471k/471k [00:13<00:00, 37.0kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName                date                 description  status    publicScore  privateScore  
----------------------  -------------------  -----------  --------  -----------  ------------  
sub57-max-c.csv         2018-12-22 01:47:16               complete  0.463        None          
sub57-max-b.csv         2018-12-22 01:45:44               complete  0.456        None          
sub57-max-a.csv         2018-12-21 23:46:32               complete  0.455        None          
sub57-max.csv           2018-12-21 08:43:25               complete  0.000        None          
sub50-max-055.csv       2018-12-18 20:29:16               complete  0.510        None          
sub50-max-f.csv         2018-12-18 20:15:51               complete  0.513        None          
sub50-max-f-1-l.csv     2018-12-18 16:40:56               complete  0.546        None

In [33]:
submit = pd.read_csv('../data/sample_submission.csv')
predicted = []
for ii, name in tqdm_notebook(enumerate(submit['Id'])):
    
    score_predict = draw_predict[ii]
    label_predict = np.arange(28)[score_predict>=0.35]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted
submit.to_csv('../submissions/sub57-max-d.csv', index=False)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [34]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub57-max-d.csv -m ""

from time import sleep
sleep(20)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 473k/473k [00:12<00:00, 37.3kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName                date                 description  status    publicScore  privateScore  
----------------------  -------------------  -----------  --------  -----------  ------------  
sub57-max-d.csv         2018-12-22 01:57:31               complete  0.464        None          
sub57-max-c.csv         2018-12-22 01:47:16               complete  0.463        None          
sub57-max-b.csv         2018-12-22 01:45:44               complete  0.456        None          
sub57-max-a.csv         2018-12-21 23:46:32               complete  0.455        None          
sub57-max.csv           2018-12-21 08:43:25               complete  0.000        None          
sub50-max-055.csv       2018-12-18 20:29:16               complete  0.510        None          
sub50-max-f.csv         2018-12-18 20:15:51               complete  0.513        None

In [35]:
# Create submit
from tqdm import tqdm_notebook
submit = pd.read_csv('../data/sample_submission.csv')
predicted = []
draw_predict = []
# model = create_model(
#     input_shape=(SIZE,SIZE,3), 
#     n_out=28)
# for layer in model.layers:
#     layer.trainable = True
# model.compile(loss=f1_loss,
#             optimizer=Adam(lr=1e-4),
#             metrics=[f1])
# model.load_weights('../cache/R50-57-maximus.h5')
for name in tqdm_notebook(submit['Id']):
    path = os.path.join('../data/test/', name)
    image = load_image2(path, (SIZE,SIZE,3))
    image = augment2(image)
    imgae = image/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.35]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted
submit.to_csv('../submissions/sub57-max-e.csv', index=False)

HBox(children=(IntProgress(value=0, max=11702), HTML(value='')))

In [37]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub57-max-e.csv -m ""

from time import sleep
sleep(20)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName             date                 description  status    publicScore  privateScore  
-------------------  -------------------  -----------  --------  -----------  ------------  
sub57-max-e.csv      2018-12-22 02:08:14               complete  0.075        None          
sub57-max-d.csv      2018-12-22 01:57:31               complete  0.464        None          
sub57-max-c.csv      2018-12-22 01:47:16               complete  0.463        None          
sub57-max-b.csv      2018-12-22 01:45:44               complete  0.456        None          
sub57-max-a.csv      2018-12-21 23:46:32               complete  0.455        None          
sub57-max.csv        2018-12-21 08:43:25               complete  0.000        None          
sub50-max-055.csv    2018-12-18 20:29:16               complete  0.510        None          
sub50-max-f.csv      2018-12-18 20:15:51               complete  0.513        None          
sub50-max-f-1-l.csv  2018-12-18 16:40:56               complete  0.546